# **Function Calling with LangChain**

In [ ]:
# Install necessary Packages
!pip install -Uq langchain-google-genai

In [ ]:
# Setup API Key
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
# Select and configure Model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key = userdata.get('GEMINI_API_KEY')
)

In [ ]:
# test if the model is configured correctly
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2bf03d64-463c-4f31-849b-1ee02a68cc5f-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
# Define Tools / Functions
from langchain_core.tools import tool
import math

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def square_rt(a: int) -> float:
    """Find square root of a number."""
    return math.sqrt(a)

@tool
def annual_return(a: int) -> float:
    """Find annual return of a number."""
    return (math.sqrt(a) * 85) / 0.15

tools = [add, multiply, square_rt, annual_return]

In [ ]:
# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from IPython.display import display

# invoke an LLM with out tool
ai_msg = llm.invoke("What is annual return of 25")
display(ai_msg)

AIMessage(content='An annual return of 25% means that an investment grew by 25% over a one-year period.  For example, if you invested $1000 and it had a 25% annual return, it would be worth $1250 at the end of the year.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-04213d5d-1b05-4525-bf1a-1b347515f513-0', usage_metadata={'input_tokens': 9, 'output_tokens': 65, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [ ]:
# invoke an LLM with tools
from langchain_core.messages import HumanMessage, AIMessage
query = "What is annual return of 25"
messages = [HumanMessage(query)]
display(messages)

[HumanMessage(content='What is annual return of 25', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_msg_tools = llm_with_tools.invoke(messages)
messages.append(ai_msg_tools)
display(messages)

[HumanMessage(content='What is annual return of 25', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-09688658-b582-4725-a4c4-e40d541c1260-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': 'd6b2a038-612d-4af3-a31c-c09cbe54412d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 4, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}})]

In [ ]:
display(ai_msg_tools)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-09688658-b582-4725-a4c4-e40d541c1260-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': 'd6b2a038-612d-4af3-a31c-c09cbe54412d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 4, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}})

In [ ]:
print(ai_msg_tools.tool_calls)

[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': 'd6b2a038-612d-4af3-a31c-c09cbe54412d', 'type': 'tool_call'}]


In [ ]:
# Invoke the function / tool
for tool_call in ai_msg_tools.tool_calls:
    selected_tool = {
        "add": add,
        "multiply": multiply,
        "square_rt":square_rt,
        "annual_return":annual_return
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    messages.append(tool_msg)

ToolMessage(content='2833.3333333333335', name='annual_return', tool_call_id='d6b2a038-612d-4af3-a31c-c09cbe54412d')

In [ ]:
display(messages)

[HumanMessage(content='What is annual return of 25', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-09688658-b582-4725-a4c4-e40d541c1260-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': 'd6b2a038-612d-4af3-a31c-c09cbe54412d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 4, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='2833.3333333333335', name='annual_return', tool_call_id='d6b2a038-612d-4af3-a31c-c09cbe54412d')]

In [ ]:
# Invoke the llm
response = llm_with_tools.invoke(messages)

In [ ]:
from IPython.display import Markdown

Markdown(response.content)

The annual return of 25 is 2833.33.